In [1]:
import pandas as pd
import re
import time
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
matches_23_24 = pd.read_csv("new_data/teams_pl_23-24_fbref.csv")
matches_22_23 = pd.read_csv("new_data/teams_pl_22-23_fbref.csv")
matches_21_22 = pd.read_csv("new_data/teams_pl_21-22_fbref.csv")
matches_20_21 = pd.read_csv("new_data/teams_pl_20-21_fbref.csv")
matches_19_20 = pd.read_csv("new_data/teams_pl_19-20_fbref.csv")
matches_18_19 = pd.read_csv("new_data/teams_pl_18-19_fbref.csv")
matches_17_18 = pd.read_csv("new_data/teams_pl_17-18_fbref.csv")
matches = pd.concat([matches_23_24, matches_22_23, matches_21_22, matches_20_21, matches_19_20, matches_18_19, matches_17_18], ignore_index=True)
matches["League"] = "Premier League"

In [3]:
matches.rename(columns={"venue_date": "date", "venue_time": "time"}, inplace=True)
matches = matches.drop(columns = ["home_shirtnumber", "home_nationality", "home_position", "home_age", "away_shirtnumber", "away_nationality", "away_position", "away_age"])

matches["home_goals"] = matches["home_goals"] + matches["away_own_goals"]
matches["away_goals"] = matches["away_goals"] + matches["home_own_goals"]
matches["total_goals"] = matches["home_goals"] + matches["away_goals"]

In [4]:
stadiums = pd.read_csv("data/stadiums.csv")
matches_with_h2h = pd.merge(matches, stadiums[stadiums["Closed"].isna()], left_on=['home_team'], right_on=['Club'], how='left').drop(columns=["Closed", "Opened"])

In [5]:
def determine_outcome_and_points(row):
    if row['home_goals'] > row['away_goals']:
        return 1, 3, 0  # Home win
    elif row['home_goals'] < row['away_goals']:
        return 2, 0, 3  # Away win
    else:
        return 0, 1, 1  # Draw  
    
matches_with_h2h[['outcome', 'home_points', 'away_points']] = matches_with_h2h.apply(lambda row: pd.Series(determine_outcome_and_points(row)), axis=1)

In [6]:
cols = matches_with_h2h.columns
attributes = [col.replace("home_", "") for col in cols if col.startswith("home_") and col != 'home_manager' and col != 'home_captain' and col != 'home_lineup' and col != 'home_team']

standings_columns = ['matches_played', 'wins', 'draws', 'defeats', 'goal_difference', 'goals_conceded'] + attributes
standings = []
matches_sorted = matches_with_h2h.sort_values(by=['season', 'date'])

for season in matches_sorted['season'].unique():
    season_data = matches_sorted[matches_sorted['season'] == season]
    teams = season_data['home_team'].unique()
    
    standings_dict = {team: {attribute: 0 for attribute in standings_columns} for team in teams}

    # tu można dodać punkty karne, z którymi drużyny zaczynają sezon, bardzo niewiele to zmieni ale jednak w teorii dokładniejszy model + żeby tabele w aplikacji się zgadzały!!!!!!
    
    for dt in season_data['date'].unique():
        round_data = season_data[season_data['date'] == dt]
        
        for index, row in round_data.iterrows():
            home_team = row['home_team']
            away_team = row['away_team']
            home_goals = row['home_goals']
            away_goals = row['away_goals']
            home_points = row['home_points']
            away_points = row['away_points']
            outcome = row['outcome']
            
            # Update home team standings
            standings_dict[home_team]['matches_played'] += 1
            standings_dict[home_team]['goals_conceded'] += away_goals
            
            for attr in attributes:
                standings_dict[home_team][attr] += row["home_" + attr]

            standings_dict[home_team]['goal_difference'] = standings_dict[home_team]['goals'] - standings_dict[home_team]['goals_conceded']
            
            # Update away team standings
            standings_dict[away_team]['matches_played'] += 1
            standings_dict[away_team]['goals_conceded'] += home_goals
            
            for attr in attributes:
                standings_dict[away_team][attr] += row["away_" + attr]

            standings_dict[away_team]['goal_difference'] = standings_dict[away_team]['goals'] - standings_dict[away_team]['goals_conceded']

            # Update W L D
            if outcome == 1:
                standings_dict[home_team]['wins'] += 1
                standings_dict[away_team]['defeats'] += 1
            elif outcome == 0:
                standings_dict[home_team]['draws'] += 1
                standings_dict[away_team]['draws'] += 1
            else:
                standings_dict[home_team]['defeats'] += 1
                standings_dict[away_team]['wins'] += 1

        temp_standings_data = {
        'season': season,
        'date': dt,
        'team': list(standings_dict.keys()),
        }
        for attr in standings_columns:
            temp_standings_data[attr] = [standings_dict[team][attr] for team in standings_dict]

        temp_standings = pd.DataFrame(temp_standings_data)

        standings.append(temp_standings)
standings = pd.concat(standings).sort_values(by=['season', 'date', 'points', 'goal_difference', 'goals'], ascending=[True, True, False, False, False]).reset_index(drop=True)
standings["League"] = "Premier League"
standings.to_csv("data/standings.csv", index=False)

In [7]:
matches_filtered = matches_with_h2h[matches_with_h2h['round'] > 1]
df = matches_filtered.copy()

def get_team_stats(season, team, matches_played):
    return standings[(standings["season"] == season) & (standings["team"] == team) & (standings["matches_played"] == matches_played)].iloc[0]


def calculate_rolling_stats(n, stats, stats_old, matches_played, real_matches_played):
    rolling_stats = {}
    for feature in standings_columns:
        if matches_played == 0:
            rolling_stats[feature] = stats[feature]
            if real_matches_played != 0:
                rolling_stats[feature] = rolling_stats[feature] / real_matches_played
        else:
            rolling_stats[feature] = stats[feature] - stats_old[feature]
            rolling_stats[feature] = rolling_stats[feature] / n
    return rolling_stats

def create_rolling_stats(n, df):
    home_stats_list = []
    away_stats_list = []
    for index, row in df.iterrows():
        date = row["date"]
        season = row["season"]
        home_team = row["home_team"]
        away_team = row["away_team"]

        home_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == home_team)]["matches_played"].values[0] - 1
        away_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == away_team)]["matches_played"].values[0] - 1

        home_team_stats = get_team_stats(season, home_team, home_team_matches_played)
        away_team_stats = get_team_stats(season, away_team, away_team_matches_played)

        home_team_matches_played_old = max(0, home_team_matches_played - n)
        away_team_matches_played_old = max(0, away_team_matches_played - n)

        if home_team_matches_played_old > 0:
            home_team_stats_old = get_team_stats(season, home_team, home_team_matches_played_old)
        if away_team_matches_played_old > 0:
            away_team_stats_old = get_team_stats(season, away_team, away_team_matches_played_old)

        home_rolling_stats = calculate_rolling_stats(n, home_team_stats, home_team_stats_old if home_team_matches_played_old > 0 else None, home_team_matches_played_old, home_team_matches_played)
        away_rolling_stats = calculate_rolling_stats(n, away_team_stats, away_team_stats_old if away_team_matches_played_old > 0 else None, away_team_matches_played_old, away_team_matches_played)

        home_stats_list.append({f"home_last{n}_{feature}": value for feature, value in home_rolling_stats.items()})
        away_stats_list.append({f"away_last{n}_{feature}": value for feature, value in away_rolling_stats.items()})

    home_stats_df = pd.DataFrame(home_stats_list, index=df.index)
    away_stats_df = pd.DataFrame(away_stats_list, index=df.index)

    return pd.concat([df, home_stats_df, away_stats_df], axis=1)

df = create_rolling_stats(n = 5, df = df)
df.to_csv("data/matches_with_rolling_stats_pl.csv", index=False)

In [8]:
matches = df
matches['date'] = pd.to_datetime(matches['date'])
matches["formation_home"] = matches["formation_home"].str.replace(r"-1-1$", "-2", regex=True)
matches["formation_away"] = matches["formation_away"].str.replace(r"-1-1$", "-2", regex=True)
matches["formation_home"] = matches["formation_home"].str.replace("4-1-2-1-2", "4-3-1-2", regex=True)
matches["formation_away"] = matches["formation_away"].str.replace("4-1-2-1-2", "4-3-1-2", regex=True)
matches["formation_back_line"] = matches["formation_home"].apply(lambda text: int(text[0] == "4"))

In [9]:
def calculate_tiredness_factor(days_since_last_match, decay_rate=0.1):
    # Higher decay rate means quicker recovery. Adjust as needed.
    return np.exp(-decay_rate * days_since_last_match)

matches['last_match_date'] = pd.NaT
matches['date'] = pd.to_datetime(matches['date'])
matches = matches.sort_values(by='date', ignore_index=True)
# Iterate through each match row
for i, row in matches.iterrows():
    # Find the home team and away team
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Initialize last match date to NaT (Not a Time) for both teams
    last_home_date = pd.NaT
    last_away_date = pd.NaT

    home_team_last_manager = None
    away_team_last_manager = None
    
    # For home team: Search for the last match where the team played (either home or away)
    for j in range(i-1, -1, -1):  # Iterate backward from the current match
        if matches.iloc[j]['home_team'] == home_team:
            last_home_date = matches.iloc[j]['date']
            home_team_last_manager = matches.iloc[j]["home_manager"]
            break  # Stop once the match is found
        if matches.iloc[j]['away_team'] == home_team:
            last_home_date = matches.iloc[j]['date']
            home_team_last_manager = matches.iloc[j]["away_manager"]
            break
    
    # For away team: Search for the last match where the team played (either home or away)
    for j in range(i-1, -1, -1):  # Iterate backward from the current match
        if matches.iloc[j]['home_team'] == away_team:
            last_away_date = matches.iloc[j]['date']
            away_team_last_manager = matches.iloc[j]["home_manager"]
            break  # Stop once the match is found
        if matches.iloc[j]['away_team'] == away_team:
            last_away_date = matches.iloc[j]['date']
            away_team_last_manager = matches.iloc[j]["away_manager"]
            break

    home_matches_since_last_manager = None
    away_matches_since_last_manager = None
    if home_team_last_manager == None:
        home_team_last_manager = row["home_manager"]
        home_matches_since_last_manager = 20
    if away_team_last_manager == None:
        away_team_last_manager = row["away_manager"]
        away_matches_since_last_manager = 20

    if home_team_last_manager == row["home_manager"]:
        new_home_manager = False
    else:
        new_home_manager = True

    if away_team_last_manager and away_team_last_manager == row["away_manager"]:
        new_away_manager = False
    else:
        new_away_manager = True

    # Assign the last match date for home and away teams
    matches.at[i, 'last_match_date_home'] = last_home_date
    matches.at[i, 'last_match_date_away'] = last_away_date
    matches.at[i, 'last_home_manager'] = home_team_last_manager
    matches.at[i, 'last_away_manager'] = away_team_last_manager
    matches.at[i, 'new_home_manager'] = new_home_manager
    matches.at[i, 'new_away_manager'] = new_away_manager
    matches.at[i, 'matches_since_new_home_manager'] = home_matches_since_last_manager
    matches.at[i, 'matches_since_new_away_manager'] = away_matches_since_last_manager    

matches['days_since_last_home'] = (matches['date'] - matches['last_match_date_home']).dt.days
matches['days_since_last_away'] = (matches['date'] - matches['last_match_date_away']).dt.days
matches['days_since_last_home'] = matches['days_since_last_home'].fillna(7)
matches['days_since_last_away'] = matches['days_since_last_away'].fillna(7)
matches['home_tiredness'] = calculate_tiredness_factor(matches['days_since_last_home'])
matches['away_tiredness'] = calculate_tiredness_factor(matches['days_since_last_away'])

In [10]:
for i in range (len(matches)):
    # Find the home team and away team
    home_team = matches.iloc[i]['home_team']
    away_team = matches.iloc[i]['away_team']

    new_home_manager = matches.iloc[i]["new_home_manager"]
    new_away_manager = matches.iloc[i]["new_away_manager"]

    if new_home_manager:
        matches.at[i, 'matches_since_new_home_manager'] = 0
    if new_away_manager:
        matches.at[i, 'matches_since_new_away_manager'] = 0

    for j in range(i+1, len(matches)):
        if matches.iloc[j]['home_team'] == home_team:
            matches.at[j, 'matches_since_new_home_manager'] = matches.iloc[i]["matches_since_new_home_manager"] + 1
            break
        if matches.iloc[j]['away_team'] == home_team:
            matches.at[j, 'matches_since_new_away_manager'] = matches.iloc[i]["matches_since_new_home_manager"] + 1
            break

    for j in range(i+1, len(matches)):
        if matches.iloc[j]['home_team'] == away_team:
            matches.at[j, 'matches_since_new_home_manager'] = matches.iloc[i]["matches_since_new_away_manager"] + 1
            break
        if matches.iloc[j]['away_team'] == away_team:
            matches.at[j, 'matches_since_new_away_manager'] = matches.iloc[i]["matches_since_new_away_manager"] + 1
            break

In [11]:
matches.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,last_home_manager,last_away_manager,new_home_manager,new_away_manager,matches_since_new_home_manager,matches_since_new_away_manager,days_since_last_home,days_since_last_away,home_tiredness,away_tiredness
0,2017-2018,2017-08-19,12:30,2,20862.0,Jonathan Moss,Paul Clement,José Mourinho,Federico Fernández,Antonio Valencia,...,Paul Clement,José Mourinho,False,False,20.0,20.0,7.0,7.0,0.496585,0.496585
1,2017-2018,2017-08-19,15:00,2,10501.0,Roger East,Eddie Howe,Marco Silva,Andrew Surman,Heurelho Gomes,...,Eddie Howe,Marco Silva,False,False,20.0,20.0,7.0,7.0,0.496585,0.496585


In [12]:
def get_h2h_metrics(row, df, num_matches=6):
    past_h2h = df[((df['home_team'] == row['home_team']) & (df['away_team'] == row['away_team'])) |
                  ((df['home_team'] == row['away_team']) & (df['away_team'] == row['home_team']))]
    past_h2h = past_h2h[past_h2h['date'] < row['date']].sort_values(by='date', ascending=False)
    
    past_h2h = past_h2h.head(num_matches)
    num_past_h2h = len(past_h2h)

    past_h2h_home_home = past_h2h[past_h2h["home_team"] == row["home_team"]]
    past_h2h_home_away = past_h2h[past_h2h["home_team"] == row["away_team"]]

    if not past_h2h.empty:
        metrics = {
            'h2h_win_ratio': ((past_h2h_home_home['outcome'] == 1).sum() + (past_h2h_home_away['outcome'] == 2).sum()) / num_past_h2h,
            'h2h_draw_ratio': (past_h2h['outcome'] == 0).sum() / num_past_h2h,
            'h2h_avg_goals_scored_home_team': (past_h2h_home_home['home_goals'].sum() + past_h2h_home_away['away_goals'].sum()) / num_past_h2h,
            'h2h_avg_goals_scored_away_team': (past_h2h_home_home['away_goals'].sum() + past_h2h_home_away['home_goals'].sum()) / num_past_h2h,
            'h2h_avg_xG_home_team': (past_h2h_home_home['home_xg'].sum() + past_h2h_home_away['away_xg'].sum()) / num_past_h2h,
            'h2h_avg_xG_away_team': (past_h2h_home_home['away_xg'].sum() + past_h2h_home_away['home_xg'].sum()) / num_past_h2h,
            # Add other metrics as needed
        }
    else:
        metrics = {
            'h2h_win_ratio': 0,
            'h2h_draw_ratio': 0,
            'h2h_avg_goals_scored_home_team': 0,
            'h2h_avg_goals_scored_away_team': 0,
            'h2h_avg_xG_home_team': 0,
            'h2h_avg_xG_away_team': 0,
        }

    return pd.Series(metrics)

matches_with_h2h = matches.join(matches.apply(lambda row: get_h2h_metrics(row, matches), axis=1))

In [13]:
matches_with_h2h.head()

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,days_since_last_home,days_since_last_away,home_tiredness,away_tiredness,h2h_win_ratio,h2h_draw_ratio,h2h_avg_goals_scored_home_team,h2h_avg_goals_scored_away_team,h2h_avg_xG_home_team,h2h_avg_xG_away_team
0,2017-2018,2017-08-19,12:30,2,20862.0,Jonathan Moss,Paul Clement,José Mourinho,Federico Fernández,Antonio Valencia,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-2018,2017-08-19,15:00,2,10501.0,Roger East,Eddie Howe,Marco Silva,Andrew Surman,Heurelho Gomes,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-2018,2017-08-19,15:00,2,31424.0,Lee Mason,Mauricio Pellegrino,Slaven Bilić,Steven Davis,Mark Noble,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-2018,2017-08-19,15:00,2,31902.0,Lee Probert,Craig Shakespeare,Chris Hughton,Wes Morgan,Bruno,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-2018,2017-08-19,15:00,2,19619.0,Martin Atkinson,Sean Dyche,Tony Pulis,Tom Heaton,Jake Livermore,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0


Dodanie informacji o pogodzie

In [15]:
def degrees_to_decimal(geo_loc):
    if pd.isna(geo_loc):
        return 0.0
    if isinstance(geo_loc, float):
        return geo_loc
    geo_loc = str(geo_loc).strip()
    pattern = r'(\d+)°\s*(\d+)′\s*(\d+(\.\d+)?)″([NSEW])'
    match = re.match(pattern, geo_loc)
    if not match:
        raise ValueError(f"Invalid format: {geo_loc}")
    degrees = int(match.group(1))
    minutes = int(match.group(2))
    seconds = float(match.group(3))
    direction = match.group(5)
    decimal_loc = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ['S', 'W']:
        decimal_loc = -decimal_loc
    return decimal_loc

In [16]:
matches_with_h2h['longitude']=matches_with_h2h['longitude'].apply(degrees_to_decimal)
matches_with_h2h['latitude']=matches_with_h2h['latitude'].apply(degrees_to_decimal)

In [17]:
from download_weather import get_weather

In [18]:
def create_weather_columns(row):
    data_weather=get_weather(
        latitude=row['latitude'],
        longitude=row['longitude'],
        start_hour=row['datetime'],
        end_hour=row['datetime'],
        params=['temperature_2m', 'precipitation', 'wind_speed_10m', 'relative_humidity_2m', 'cloud_cover'],
        type_url='historical'
    )
    row['weather_temperature']=data_weather['temperature_2m'][0]
    row['weather_precipitation']=data_weather['precipitation'][0]
    row['weather_wind']=data_weather['wind_speed_10m'][0]
    row['weather_humidity']=data_weather['relative_humidity_2m'][0]
    row['weather_cloud_cover']=data_weather['cloud_cover'][0]
    return row

In [19]:
matches_with_h2h['datetime'] = pd.to_datetime(matches_with_h2h['date'].astype(str) + ' ' + matches_with_h2h['time'].astype(str))
matches_with_h2h['datetime'] = matches_with_h2h['datetime'].dt.strftime('%Y-%m-%dT%H:%M')
matches_with_h2h.insert(1, 'datetime', matches_with_h2h.pop('datetime'))
matches_with_h2h.drop(columns=['date', 'time'], inplace=True)
matches_with_h2h.head()

,season,datetime,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,...,days_since_last_home,days_since_last_away,home_tiredness,away_tiredness,h2h_win_ratio,h2h_draw_ratio,h2h_avg_goals_scored_home_team,h2h_avg_goals_scored_away_team,h2h_avg_xG_home_team,h2h_avg_xG_away_team
0,2017-2018,2017-08-19T12:30,2,20862.0,Jonathan Moss,Paul Clement,José Mourinho,Federico Fernández,Antonio Valencia,3-1-4-2,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-2018,2017-08-19T15:00,2,10501.0,Roger East,Eddie Howe,Marco Silva,Andrew Surman,Heurelho Gomes,4-4-2,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-2018,2017-08-19T15:00,2,31424.0,Lee Mason,Mauricio Pellegrino,Slaven Bilić,Steven Davis,Mark Noble,4-2-3-1,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-2018,2017-08-19T15:00,2,31902.0,Lee Probert,Craig Shakespeare,Chris Hughton,Wes Morgan,Bruno,4-4-2,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-2018,2017-08-19T15:00,2,19619.0,Martin Atkinson,Sean Dyche,Tony Pulis,Tom Heaton,Jake Livermore,4-4-2,...,7.0,7.0,0.496585,0.496585,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
matches_with_h2h=matches_with_h2h.apply(create_weather_columns, axis=1)

In [24]:
matches_with_h2h.to_csv('matches_with_weather.csv')

In [84]:
def split_and_exact_match(short_name, long_name):
    short_parts = re.split(r'[ \-]', short_name)
    exact_matches = [part for part in short_parts if part in long_name]
    return len(exact_matches) == len(short_parts)

def longest_two_substrings(s1, s2):
    # Initialize the matrix for storing common substring lengths
    matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    longest_substrings = []
    
    # Populate the matrix
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                matrix[i][j] = matrix[i - 1][j - 1] + 1
                if matrix[i][j] > 1:  # Only consider substrings longer than 1
                    longest_substrings.append(s1[i - matrix[i][j]:i])
            else:
                matrix[i][j] = 0

    # Sort substrings by length and pick the top 2 longest ones
    unique_substrings = sorted(set(longest_substrings), key=len, reverse=True)
    return unique_substrings[:2]
def similarity_score(name1, name2):
    if split_and_exact_match(name1, name2):
        return 100
    else:
        substrings = longest_two_substrings(name1, name2)
        return sum(len(sub) for sub in substrings)
def find_best_match_with_longest_substrings(name, choices):
    scores = {choice: similarity_score(name, choice) for choice in choices}
    best_match = max(scores, key=scores.get)  # Find the match with the highest score
    return best_match
fifa_names = fifa["name"].unique()
player_names = players["player"].unique()

mapping_dict = {
    player_name: find_best_match_with_longest_substrings(player_name, fifa_names)
    for player_name in player_names
}

In [86]:
players["player_name"] = players["player"].map(mapping_dict)
players['date'] = pd.to_datetime(players['date'])
players_joined = pd.merge(players, fifa, left_on=["player_name","season"], right_on=["name", "season"], how="left")

for idx, row in players_joined[players_joined["name"].isnull()].iterrows():
    matching = fifa[fifa["name"] == row["player_name"]]
    if not matching.empty:
        for col in matching.columns:
            players_joined.loc[idx, col] = matching.iloc[0][col]

In [87]:
def get_starters(group):
    starters = []
    group = group.sort_index()
    used_indices = set()
    for idx, row in group.iterrows():
        if idx in used_indices:
            continue

        if row['minutes'] == 90:
            starters.append(group.index.get_loc(idx))
            used_indices.add(idx)
        elif row['minutes'] < 90:
            starters.append(group.index.get_loc(idx))
            used_indices.add(idx)
            minutes_sum = row['minutes']
            next_row = row
            next_idx_global = idx
            while minutes_sum < 90 and next_row['cards_red'] < 1:
                next_idx = group.index.get_loc(next_idx_global) + 1
                next_idx_global = next_idx_global + 1
                if next_idx < len(group):
                    next_row = group.iloc[next_idx]
                    minutes_sum += next_row['minutes']
                    if minutes_sum > 91:
                        starters.append(next_idx)
                    used_indices.add(next_idx_global)
                else:
                    minutes_sum = 90
                    
    group = group.iloc[starters]
    return group

In [93]:
player_stat_columns = [col for col in players_joined.columns if col.startswith('stat_') or col == "overall_rating"]
# player_stat_columns = ["overall_rating"]
for _, match in matches.iterrows():
    match_date = match['date']
    home_team = match['home_team']
    away_team = match['away_team']
    
    home_players = get_starters(players_joined[(players_joined['team'] == home_team) & (players_joined['date'] == match_date)])
    away_players = get_starters(players_joined[(players_joined['team'] == away_team) & (players_joined['date'] == match_date)])
    
    home_stats_avg = home_players[player_stat_columns].mean()
    away_stats_avg = away_players[player_stat_columns].mean()
    
    for stat in player_stat_columns:
        matches.at[_, f'overall_home_{stat}'] = home_stats_avg[stat]
        matches.at[_, f'overall_away_{stat}'] = away_stats_avg[stat]

In [94]:
matches.to_csv("data/prepared_data_pl.csv", index=False)

In [95]:
matches.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_possession,away_possession,home_lineup,away_lineup,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,League,total_goals,Stadium,